### Do required imports

In [56]:
from __future__ import print_function
import tweepy
import json
from pymongo import MongoClient

### Setup your mongo path

In [57]:
MONGO_HOST = 'mongodb://localhost/twitterdb'

### Prepare a search keywords

In [58]:
WORDS =['#bigdata','#AI','#datascience','#machinelearning','#ml','#iot']

### Set up Twitter API tokens and consumder keys

In [4]:
CONSUMER_KEY='vwCAn6JbypbkrVXOBMbMrqQvO'
CONSUMER_SECRET='oqltY6Xls78JhS37sss10ZW6oKBIyPIqyw9ZStzwhVWuhVl6gv'
ACCESS_KEY='1337046363838828545-Dh901imr730DwV9VDKq3XVxRnWYqHU'
ACCESS_SECRET='LSsztMDGPhoTeBOoS284i4kESzM4p2zsqOIx7tFopKjNG'

In [5]:
class StreamListener(tweepy.StreamListener):
    # This is the class provided by tweepy to access twitter Streaming API

    def on_connect(self):
        # called initially to connect to the streaming API
        print("you are connected to the streaming API")
    
    
    def on_error(self,status_code):
        # on error - if an error occurs displays the error / status code
        print("An Error has occured: " + repr(status_code))
        return False
    
    def on_data(self,data):
        # this part is main script where we connect to MongoDB and stores the tweet
        try:
            client = MongoClient(MONGO_HOST)
            
            # use twitterdb database. if it doesn't existed, it will be created
            db= client.twitterdb
            
            # Decode the JSON response from Twitter
            datajson = json.loads(data)
            
            # grab the Created_at data from the tweet to use for display
            created_at = datajson['created_at']
            
            # print out the message on every successful tweet we collected
            print("Tweet collected at " + str(created_at))
            
            # insert the data into the mongodb collection called twitter_search
            # it twitter_search is not existed, it will be created
            db.twitter_search.insert(datajson)
        
        except Exception as e:
            print(e)

        
        

In [ ]:
 # authentification so we can access twitter
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
api =tweepy.API(auth, wait_on_rate_limit=True)

# create instance of Streamlistener
listener = StreamListener(api = api)
stream = tweepy.Stream(auth, listener = listener)

#track = ['golf', 'masters', 'reed', 'mcilroy', 'woods']
#track = ['nba', 'cavs', 'celtics', 'basketball']
# choose what we want to filter by
#print('Tracking: ' +str(WORDS))
stream.filter(track = WORDS,languages=['en'])

### Read data from mongoDB

In [59]:
import pymongo
import pandas as pd
from pymongo import MongoClient

In [60]:
client = MongoClient("mongodb://localhost:27017")

In [61]:
db = client['twitter_db']

In [62]:
#db.tweets.drop()

In [63]:
tweets = db['twitter_search']

In [64]:
df = pd.DataFrame(list(tweets.find()))

In [65]:
df.head()

,_id,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,lang,timestamp_ms,display_text_range,extended_tweet,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,withheld_in_countries
0,5fd512ae1c612b9b42dd53f9,Sat Dec 12 18:57:45 +0000 2020,1337834038262829056,1337834038262829056,RT @SourabhSKatoch: Daily Life as a Data Scien...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,...,en,1607799465133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5fd512b01c612b9b42dd53fb,Sat Dec 12 18:57:47 +0000 2020,1337834046794117120,1337834046794117120,RT @lopezunwired: Google used a 64-camera rig ...,,False,NaN,None,NaN,...,en,1607799467167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5fd512b01c612b9b42dd53fd,Sat Dec 12 18:57:47 +0000 2020,1337834048690024453,1337834048690024453,RT @SourabhSKatoch: Daily Life as a Data Scien...,"<a href=""https://nlogn.in"" rel=""nofollow"">Nlog...",False,NaN,None,NaN,...,en,1607799467619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5fd512b11c612b9b42dd53ff,Sat Dec 12 18:57:47 +0000 2020,1337834049503629312,1337834049503629312,RT @GiovannaDiTroia: L'apprendimento automatic...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,...,it,1607799467813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5fd512b31c612b9b42dd5401,Sat Dec 12 18:57:50 +0000 2020,1337834059930755075,1337834059930755075,RT @jamyjake1: #coding #codenewbie #100DaysOfM...,,False,NaN,None,NaN,...,und,1607799470299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# Drop unwanted columns
list_unwanted_col = ['_id','id','truncated','in_reply_to_status_id','in_reply_to_status_id_str','in_reply_to_user_id','in_reply_to_user_id_str',\
                    'in_reply_to_screen_name','geo','coordinates','retweet_count','place','contributors','is_quote_status','extended_tweet','quote_count','reply_count',\
                    'favorite_count','entities','favorited','retweeted','possibly_sensitive','filter_level','timestamp_ms','quoted_status_id',\
                    'quoted_status_id_str','quoted_status','quoted_status_permalink','retweeted_status','display_text_range','extended_entities', 'withheld_in_countries', 'created_at', 'source']
new_df = df.drop(list_unwanted_col,axis=1)

In [69]:
new_df.head()

,id_str,text,user,lang
0,1337834038262829056,RT @SourabhSKatoch: Daily Life as a Data Scien...,"{'id': 1247524865566838784, 'id_str': '1247524...",en
1,1337834046794117120,RT @lopezunwired: Google used a 64-camera rig ...,"{'id': 1331034000404647936, 'id_str': '1331034...",en
2,1337834048690024453,RT @SourabhSKatoch: Daily Life as a Data Scien...,"{'id': 1275056773443973124, 'id_str': '1275056...",en
3,1337834049503629312,RT @GiovannaDiTroia: L'apprendimento automatic...,"{'id': 1100854040290828288, 'id_str': '1100854...",it
4,1337834059930755075,RT @jamyjake1: #coding #codenewbie #100DaysOfM...,"{'id': 1331034000404647936, 'id_str': '1331034...",und


In [70]:
user_df = pd.DataFrame.from_dict([new_df['user'][i] for i in range(len(new_df['user']))])

In [71]:
user_df.columns

Index(['id', 'id_str', 'name', 'screen_name', 'location', 'url', 'description',
       'translator_type', 'protected', 'verified', 'followers_count',
       'friends_count', 'listed_count', 'favourites_count', 'statuses_count',
       'created_at', 'utc_offset', 'time_zone', 'geo_enabled', 'lang',
       'contributors_enabled', 'is_translator', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image',
       'profile_image_url', 'profile_image_url_https', 'default_profile',
       'default_profile_image', 'following', 'follow_request_sent',
       'notifications', 'profile_banner_url'],
      dtype='object')

In [72]:
user_df

,id,id_str,name,screen_name,location,url,description,translator_type,protected,verified,...,profile_text_color,profile_use_background_image,profile_image_url,profile_image_url_https,default_profile,default_profile_image,following,follow_request_sent,notifications,profile_banner_url
0,1247524865566838784,1247524865566838784,Instatechnews,instatechnews,None,None,A tech blog with an aim to help people decide ...,none,False,False,...,333333,True,http://abs.twimg.com/sticky/default_profile_im...,https://abs.twimg.com/sticky/default_profile_i...,True,False,None,None,None,NaN
1,1331034000404647936,1331034000404647936,MedAI Bot,medaibot,None,https://Cardiologytimes.today,Created by Doctor @KrittanawongMD. A bot that ...,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1331276094...,https://pbs.twimg.com/profile_images/133127609...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/13310340...
2,1275056773443973124,1275056773443973124,Jessica shoonover,Women_who_code,bootcamp,None,24 🧬 ~ Javascript ❤️ ~ Mern Stack Developer ~ ...,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1319806256...,https://pbs.twimg.com/profile_images/131980625...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/12750567...
3,1100854040290828288,1100854040290828288,CarloB.,CarloB30482104,None,None,La voglia di vincere non è nemmeno lontanament...,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1100854725...,https://pbs.twimg.com/profile_images/110085472...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/11008540...
4,1331034000404647936,1331034000404647936,MedAI Bot,medaibot,None,https://Cardiologytimes.today,Created by Doctor @KrittanawongMD. A bot that ...,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1331276094...,https://pbs.twimg.com/profile_images/133127609...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/13310340...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118470,338794637,338794637,Dawn PurcellMusgrave,gonzo_30531,FL,None,poshmark/@samipoo24,none,False,False,...,000000,False,http://pbs.twimg.com/profile_images/1303332744...,https://pbs.twimg.com/profile_images/130333274...,False,False,None,None,None,https://pbs.twimg.com/profile_banners/33879463...
118471,1288887110943027201,1288887110943027201,Blackstembot,blackstembot,"Florida, USA",http://instagram.com/sayhitojustin,"Dad= @sayhitojustin1😷🤖Teaching him #Japanese, ...",none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1326727704...,https://pbs.twimg.com/profile_images/132672770...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/12888871...
118472,3040871649,3040871649,HubOfML,hubofml,Germany,http://bit.ly/hubofml1,"Once a month, you'll get a newsletter containi...",none,False,False,...,000000,False,http://pbs.twimg.com/profile_images/1254334743...,https://pbs.twimg.com/profile_images/125433474...,False,False,None,None,None,https://pbs.twimg.com/profile_banners/30408716...
118473,792530043390099456,792530043390099456,StarlordTechnologies,starlordtech,41 Ndidem Usang iso Marian,http://www.starlordtechnologies.com,Starlord Technologies is a next-generation glo...,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1142187029...,https://pbs.twimg.com/profile_images/114218702...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/79253004...


In [73]:
user_df['id_str'] = new_df['id_str']

In [74]:
user_df.columns

Index(['id', 'id_str', 'name', 'screen_name', 'location', 'url', 'description',
       'translator_type', 'protected', 'verified', 'followers_count',
       'friends_count', 'listed_count', 'favourites_count', 'statuses_count',
       'created_at', 'utc_offset', 'time_zone', 'geo_enabled', 'lang',
       'contributors_enabled', 'is_translator', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image',
       'profile_image_url', 'profile_image_url_https', 'default_profile',
       'default_profile_image', 'following', 'follow_request_sent',
       'notifications', 'profile_banner_url'],
      dtype='object')

In [75]:
user_df.head()

,id,id_str,name,screen_name,location,url,description,translator_type,protected,verified,...,profile_text_color,profile_use_background_image,profile_image_url,profile_image_url_https,default_profile,default_profile_image,following,follow_request_sent,notifications,profile_banner_url
0,1247524865566838784,1337834038262829056,Instatechnews,instatechnews,None,None,A tech blog with an aim to help people decide ...,none,False,False,...,333333,True,http://abs.twimg.com/sticky/default_profile_im...,https://abs.twimg.com/sticky/default_profile_i...,True,False,None,None,None,NaN
1,1331034000404647936,1337834046794117120,MedAI Bot,medaibot,None,https://Cardiologytimes.today,Created by Doctor @KrittanawongMD. A bot that ...,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1331276094...,https://pbs.twimg.com/profile_images/133127609...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/13310340...
2,1275056773443973124,1337834048690024453,Jessica shoonover,Women_who_code,bootcamp,None,24 🧬 ~ Javascript ❤️ ~ Mern Stack Developer ~ ...,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1319806256...,https://pbs.twimg.com/profile_images/131980625...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/12750567...
3,1100854040290828288,1337834049503629312,CarloB.,CarloB30482104,None,None,La voglia di vincere non è nemmeno lontanament...,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1100854725...,https://pbs.twimg.com/profile_images/110085472...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/11008540...
4,1331034000404647936,1337834059930755075,MedAI Bot,medaibot,None,https://Cardiologytimes.today,Created by Doctor @KrittanawongMD. A bot that ...,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1331276094...,https://pbs.twimg.com/profile_images/133127609...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/13310340...


In [80]:
user_unwanted_col = ['id', 'url','translator_type','protected','verified','utc_offset','time_zone','geo_enabled',\
                    'lang','contributors_enabled','is_translator','profile_background_color','profile_background_image_url',\
                     'profile_background_image_url_https','profile_background_tile', 'profile_link_color',\
                     'profile_sidebar_border_color', 'profile_sidebar_fill_color','profile_text_color', 'profile_use_background_image',\
                     'profile_banner_url','default_profile', 'default_profile_image', 'following','follow_request_sent', 'notifications','profile_image_url','profile_image_url_https']

In [81]:
clean_user_df = user_df.drop(user_unwanted_col,axis=1)
clean_user_df

,id_str,name,screen_name,location,description,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at
0,1337834038262829056,Instatechnews,instatechnews,None,A tech blog with an aim to help people decide ...,12,33,0,246,195,Tue Apr 07 14:03:23 +0000 2020
1,1337834046794117120,MedAI Bot,medaibot,None,Created by Doctor @KrittanawongMD. A bot that ...,73,4,2,0,5660,Tue Nov 24 00:36:55 +0000 2020
2,1337834048690024453,Jessica shoonover,Women_who_code,bootcamp,24 🧬 ~ Javascript ❤️ ~ Mern Stack Developer ~ ...,1665,23,30,455,199485,Mon Jun 22 13:23:34 +0000 2020
3,1337834049503629312,CarloB.,CarloB30482104,None,La voglia di vincere non è nemmeno lontanament...,122,421,2,6488,5044,Wed Feb 27 20:23:50 +0000 2019
4,1337834059930755075,MedAI Bot,medaibot,None,Created by Doctor @KrittanawongMD. A bot that ...,73,4,2,0,5661,Tue Nov 24 00:36:55 +0000 2020
...,...,...,...,...,...,...,...,...,...,...,...
118470,1338585941032787969,Dawn PurcellMusgrave,gonzo_30531,FL,poshmark/@samipoo24,1762,1447,68,114544,319279,Wed Jul 20 03:11:39 +0000 2011
118471,1338585945063493632,Blackstembot,blackstembot,"Florida, USA","Dad= @sayhitojustin1😷🤖Teaching him #Japanese, ...",207,30,7,14635,50860,Thu Jul 30 17:20:30 +0000 2020
118472,1338585969390456840,HubOfML,hubofml,Germany,"Once a month, you'll get a newsletter containi...",6865,12,152,32092,299756,Mon Feb 16 15:33:55 +0000 2015
118473,1338585980115234825,StarlordTechnologies,starlordtech,41 Ndidem Usang iso Marian,Starlord Technologies is a next-generation glo...,45,185,0,602,91,Sun Oct 30 00:54:21 +0000 2016


In [84]:
clean_user_df=clean_user_df.dropna(subset=['location'])
clean_user_df

,id_str,name,screen_name,location,description,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at
2,1337834048690024453,Jessica shoonover,Women_who_code,bootcamp,24 🧬 ~ Javascript ❤️ ~ Mern Stack Developer ~ ...,1665,23,30,455,199485,Mon Jun 22 13:23:34 +0000 2020
6,1337834060350164993,FemTech_🖥👩🏽‍💻,femtech_,"Berlin, Deutschland",I'm a friendly bot retweeting female developer...,16960,2,186,87,950486,Tue Jun 25 14:36:58 +0000 2019
8,1337834062610886662,HubOfML,hubofml,Germany,"Once a month, you'll get a newsletter containi...",6850,12,150,32093,298035,Mon Feb 16 15:33:55 +0000 2015
10,1337834065240731655,NerdyFlow,nerdyflow,"Nevada, USA",Draw your imagination. The leading graphic tab...,12703,2728,47,270716,567214,Tue Mar 26 04:19:19 +0000 2019
14,1337834110685995012,Gatuaprof,gatuaprof,Greenbay,"Biomedical scientists-molecular biology, virol...",217,601,1,750,249,Sun Apr 28 13:13:33 +0000 2019
...,...,...,...,...,...,...,...,...,...,...,...
118470,1338585941032787969,Dawn PurcellMusgrave,gonzo_30531,FL,poshmark/@samipoo24,1762,1447,68,114544,319279,Wed Jul 20 03:11:39 +0000 2011
118471,1338585945063493632,Blackstembot,blackstembot,"Florida, USA","Dad= @sayhitojustin1😷🤖Teaching him #Japanese, ...",207,30,7,14635,50860,Thu Jul 30 17:20:30 +0000 2020
118472,1338585969390456840,HubOfML,hubofml,Germany,"Once a month, you'll get a newsletter containi...",6865,12,152,32092,299756,Mon Feb 16 15:33:55 +0000 2015
118473,1338585980115234825,StarlordTechnologies,starlordtech,41 Ndidem Usang iso Marian,Starlord Technologies is a next-generation glo...,45,185,0,602,91,Sun Oct 30 00:54:21 +0000 2016


In [85]:
# Merge new_df clean_user_df
combined_df = clean_user_df.merge(new_df,on='id_str')
combined_df.columns

Index(['id_str', 'name', 'screen_name', 'location', 'description',
       'followers_count', 'friends_count', 'listed_count', 'favourites_count',
       'statuses_count', 'created_at', 'text', 'user', 'lang'],
      dtype='object')

In [89]:
combined_df=combined_df.drop(columns=['user'])

In [90]:
combined_df.head()

,id_str,name,screen_name,location,description,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at,text,lang
0,1337834048690024453,Jessica shoonover,Women_who_code,bootcamp,24 🧬 ~ Javascript ❤️ ~ Mern Stack Developer ~ ...,1665,23,30,455,199485,Mon Jun 22 13:23:34 +0000 2020,RT @SourabhSKatoch: Daily Life as a Data Scien...,en
1,1337834060350164993,FemTech_🖥👩🏽‍💻,femtech_,"Berlin, Deutschland",I'm a friendly bot retweeting female developer...,16960,2,186,87,950486,Tue Jun 25 14:36:58 +0000 2019,RT @SourabhSKatoch: Daily Life as a Data Scien...,en
2,1337834062610886662,HubOfML,hubofml,Germany,"Once a month, you'll get a newsletter containi...",6850,12,150,32093,298035,Mon Feb 16 15:33:55 +0000 2015,RT @SourabhSKatoch: Daily Life as a Data Scien...,en
3,1337834065240731655,NerdyFlow,nerdyflow,"Nevada, USA",Draw your imagination. The leading graphic tab...,12703,2728,47,270716,567214,Tue Mar 26 04:19:19 +0000 2019,RT @Sprite4it: Best 4 Medical Google Slides Te...,en
4,1337834110685995012,Gatuaprof,gatuaprof,Greenbay,"Biomedical scientists-molecular biology, virol...",217,601,1,750,249,Sun Apr 28 13:13:33 +0000 2019,@NjeriAquila @Ruthnanje @RitahNabunje @EvaAkurut,und


In [91]:
# identify lat & lang based on location column
user_locations = combined_df['location'].str.replace(' ','')
user_locations

0                      bootcamp
1            Berlin,Deutschland
2                       Germany
3                    Nevada,USA
4                      Greenbay
                  ...          
76885                        FL
76886               Florida,USA
76887                   Germany
76888    41NdidemUsangisoMarian
76889                     Ghana
Name: location, Length: 76890, dtype: object

In [ ]:
from config import gkey
import requests
lat=[]
lng=[]

for location in user_locations:
    
    #Build and end point URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(location, gkey)
    
    print(target_url)
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
    
    try:
        # Extract latitude and longitude
        lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
        lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])
    except:
        print(location)
        pass

In [94]:
# Initialaizing Lat and Lng with identied values for each country to avoid running against Google API
latitude = [40.7324194, 40.7324194, 36.1176734, 40.7324194, 40.7324194, 40.7324194, 52.52000659999999, 40.7324194, 51.165691, 40.7324194, 38.8026097, 40.7324194, 40.7324194, 40.7324194, 44.5133188, 40.7324194, 36.0637737, 40.46366700000001, 40.7324194, 40.7324194, 38.8026097, 40.7324194, 40.7324194, 46.5088803, 34.0522342, 36.1176734, 40.7324194, 28.7040592, 40.7324194, 47.6062095, 32.844017, 36.1176734, 14.9798997, 41.8781136, 34.0522342, 38.8026097, 32.844017, 37.2321077, 40.7324194, 40.7324194, 37.2707022, 32.844017, 35.70145979999999, 40.7324194, 36.1176734, 37.1613814, 37.2321077, 35.0182505, 52.3555177, 20.593684, 20.593684, 51.5073509, 20.593684, 40.7324194, 56.130366, 52.6320478, 56.130366, 20.593684, 40.7324194, 37.2321077, 35.6414529, 20.593684, 28.7040592, 40.7324194, 37.0373074, 40.7324194, 40.7324194, 37.09024, 37.09024, 44.09219299999999, 40.7324194, 27.6648274, 40.7324194, 40.7324194, 36.09985959999999, 40.7324194, 39.101825, 40.7324194, 59.32932349999999, 35.6940309, 45.5051064, 37.3541079, 60.12816100000001, 40.7324194, 43.296482, 40.7324194, 46.818188, 40.7324194, 40.7324194, 52.3555177, 42.88644679999999, 40.7324194, 36.1699412, 18.5204303, 40.7324194, -23.5505199, 44.977753, 39.9525839, 37.0373074, 48.856614, 36.1176734, 40.7324194, 41.153332, 26.2297398, 40.7324194, 43.15657789999999, 52.132633, 7.873053999999999, 51.5073509, 51.5073509, 51.5073509, 40.7324194, 51.5073509, 51.5073509, 51.5073509, 51.5073509, 37.42410599999999, 51.5073509, 51.5073509, 51.5073509, 40.7324194, 42.3736158, 40.7830603, 37.2321077, 52.52000659999999, 40.7324194, 52.3555177, 40.7836826, 51.165691, 37.09024, 40.7324194, 40.7324194, -0.023559, 37.0373074, 37.0373074, 51.165691, 25.7616798, 51.165691, 39.101825, 41.909611, 40.7324194, 43.653226, 40.7324194, 40.7324194, 20.593684, 55.378051, 52.3555177, 19.0759837, 52.3675734, 40.7324194, 40.7324194, 40.7324194, 52.52000659999999, 51.5073509, 51.5073509, 51.5073509, 40.7324194, 17.385044, 40.7324194, 43.67771760000001, 40.7324194, 18.5204303, 40.7324194, 40.7324194, 40.7348417, 52.3555177, 14.554729, 40.7476472, 38.9071923, 40.7324194, 39.7392358, 40.7324194, 40.7324194, 40.7324194, 37.7749295, 38.9784551, 38.9784551, 51.48158100000001, 40.7324194, 52.52000659999999, 40.7324194, 40.4167754, 30.6295105, 40.7324194, 36.1626638, 40.7324194, 47.9955824, -0.023559, 40.7324194, 19.0330488, 40.7324194, 28.7040592, 1.352083, 51.165691, 40.7324194, 40.7324194, 40.7127753, 52.3555177, 23.022505, 40.7324194, 36.9308935, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 52.3555177, 33.7174708, 41.3850639, 40.7324194, 40.7324194, 46.818188, 50.8503463, 38.9784551, 38.9784551, 40.7324194, 40.7324194, 40.7324194, -25.274398, 40.7324194, 36.1019219, 36.1019219, 8.537981, -1.2920659, 40.7324194, 56.8430993, 38.8026097, 22.572646, 37.09024, 42.844668, 40.7324194, 51.165691, 29.9932582, 56.49067119999999, 40.7324194, 40.7324194, 40.1570707, 48.1351253, 51.165691, 40.7324194, 40.1570707, 48.0987106, 40.7324194, 42.966185, 9.081999, 40.4167754, 28.7040592, -0.023559, 40.7324194, 40.7324194, 40.7324194, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 37.0373074, 37.0373074, 37.0373074, 37.0373074, 37.0373074, 37.0373074, 37.0373074, 37.0373074, 37.0373074, 37.0373074, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 39.0457549, 40.7324194, 40.7324194, 37.0373074, 37.0373074, 37.0373074, 40.7324194, 52.3675734, 40.7324194, 40.7324194, 20.593684, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 20.593684, 29.7604267, 33.4483771, 33.4483771, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 42.0307812, 36.1486659, 40.7324194, 36.0637737, 40.4167754, 28.7040592, 29.948703, 40.7324194, 27.0238036, 37.09024, 37.2321077, 40.7324194, 51.165691, 40.7324194, 37.2321077, 20.9673702, -37.8136276, 40.7324194, 37.2321077, 30.267153, 30.267153, 30.267153, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 3.202778, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 46.818188, 35.8898314, 37.2321077, 36.092086, 37.0373074, 20.593684, 36.1176734, 20.593684, 51.165691, 40.7324194, 37.2321077, 37.0373074, 40.7324194, 40.7324194, 40.7324194, 40.7127753, 40.7127753, 9.081999, 37.6607746, 40.7324194, 41.3850639, 9.081999, 40.7324194, 9.081999, 9.081999, 40.7324194, 36.9308935, 51.5073509, 51.5073509, 40.7324194, 46.227638, 56.130366, 47.9955824, 9.081999, 42.7875235, 43.296482, 39.2777802, 40.7324194, 36.1539816, 40.7324194, 40.7324194, 7.873053999999999, 40.7324194, 40.7324194, 42.3930959, 37.2321077, 37.0373074, 52.52000659999999, 40.7324194, 34.0522342, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 6.5243793, 40.7324194, 37.3382082, 40.7324194, 6.48349, 38.423734, 46.862496, 40.7324194, 34.0522342, 6.5243793, 40.1570707, 25.7616798, 41.909611, 36.4167498, 23.634501, 48.900552, 46.818188, 34.0522342, 48.856614, 47.3768866, 36.1176734, 52.132633, 20.593684, 46.2043907, 38.9071923, 55.378051, 9.081999, 37.09024, 43.6429251, 40.1570707, 37.2321077, 40.7607793, 43.6429251, 37.09024, 40.7324194, 37.7750787, 41.8526335, 6.5243793, 37.7750787, 39.9525839, 19.4326077, 40.7324194, 40.7324194, 36.1176734, 40.7324194, 36.1539816, 40.7324194, 7.873053999999999, 40.7324194, 37.09024, 40.7324194, 40.7324194, 40.7324194, 37.7750787, 43.4642578, 40.7324194, 40.7324194, 40.7324194, 7.873053999999999, 6.5243793, 40.7324194, 40.7324194, 46.862496, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.1570707, 40.1570707, 52.52000659999999, 6.5243793, 19.4326077, 52.52000659999999, 40.7324194, 55.755826, 50.1109221, 51.5073509, 40.7324194, 40.7324194, 40.7324194, 52.52000659999999, 51.165691, 40.7324194, 7.873053999999999, 40.7324194, 52.52000659999999, 31.046051, 40.7324194, 40.7324194, 40.0421401, 40.7324194, 40.7324194, 52.52000659999999, 40.7324194, 40.7324194, 51.165691, 43.7695604, 40.7324194, 37.2321077, 39.51291, 40.7324194, 40.7324194, 43.653226, 40.7324194, 36.1176734, 42.8105356, 42.8105356, 46.818188, 40.7324194, 3.1396258, 40.7324194, 37.2321077, 37.5796973, 40.7324194, 42.8105356, 40.7324194, 7.873053999999999, 40.7324194, 46.3115038, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 52.52000659999999, 35.0843859, 40.7324194, 38.9784551, 40.0873718, 41.6763545, 6.5243793, 40.7324194, 40.7324194, 37.3382082, 40.7324194, 36.1176734, 7.873053999999999, 40.7324194, 40.7324194, 40.7324194, 52.52000659999999, 40.46366700000001, 6.42375, 40.7324194, 37.0373074, 37.09024, 34.0522342, 37.09024, 46.818188, 34.0522342, 37.09024, 36.1176734, 37.09024, 46.862496, 51.165691, 37.09024, 35.9943032, 41.3850639, 37.09024, 36.1699412, 37.2321077, 39.0457549, 42.3736158, 40.7324194, 34.0522342, 34.0522342, 37.3382082, 40.7324194, 35.9943032, 46.862496, 46.862496, 40.7324194, 28.7040592, 33.4483771, 37.2321077, 32.715738, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 46.3115038, 40.7324194, 35.9943032, 40.5852602, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 36.9308935, 40.7324194, 55.378051, 7.873053999999999, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 37.2321077, 40.7324194, 40.7324194, 40.7324194, 52.52000659999999, 46.862496, 40.7324194, 4.710988599999999, 46.862496, 40.7324194, 40.7324194, 41.1579438, 40.7324194, 40.7324194, 41.1579438, 40.7324194, 47.9955824, 20.593684, 40.7324194, 37.2321077, 40.7324194, 40.7324194, 46.862496, 40.7324194, 37.3382082, 40.7324194, 41.153332, 37.4315734, 40.7324194, 37.2321077, 40.7324194, 6.5243793, 40.7324194, 32.844017, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 32.844017, 40.7324194, 41.87194, 40.7324194, 46.862496, 51.5073509, 51.5073509, 53.9332706, 40.7324194, 39.0457549, 41.87194, 40.7324194, 36.778261, 6.5243793, 40.7127753, 40.7324194, 40.7324194, 41.87194, 40.7324194, 51.165691, 41.87194, 40.7127753, 40.7324194, 40.7324194, 36.1176734, 34.0522342, 34.0522342, 40.7324194, 46.227638, 40.7324194, 39.0457549, 46.862496, 41.3850639, 13.0826802, 34.0522342, 50.1109221, 40.7324194, 34.0522342, 6.5243793, 40.46366700000001, 37.3382082, 42.4958417, 42.4958417, 40.7324194, 41.8780025, 28.6139391, 40.0421401, 40.7324194, 40.7324194, 40.46366700000001, 40.7324194, 51.05769480000001, 13.0826802, 40.7324194, 40.7564269, 34.0522342, 40.7324194, 3.202778, 44.837789, 50.1109221, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 42.4958417, 34.0522342, 40.7324194, 39.768403, 9.081999, 42.4958417, 46.862496, -40.900557, 51.165691, 40.1570707, 46.862496, 46.818188, 40.7324194, 42.4958417, 39.0457549, 35.7795897, 40.7324194, 40.7324194, -0.1806532, 37.2321077, 38.3498195, 40.7324194, 40.7324194, 28.6139391, 40.7324194, 40.7324194, 40.7324194, 6.5243793, -25.274398, 40.7324194, 40.7324194, 51.209348, 20.6596988, 40.7324194, 6.5243793, 40.7324194, 40.7324194, 48.856614, 43.296482, 17.385044, 40.7324194, 46.862496, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 6.5243793, 46.862496, 41.8085208, 12.9715987, 12.9715987, 40.7324194, 40.7324194, 41.3850639, 48.10807699999999, 40.7324194, 33.5678062, 41.8085208, 46.3115038, 37.09024, 37.09024, 37.09024, 40.7324194, 37.0373074, 40.7324194, 40.7324194, 39.101825, 40.7324194, 46.862496, 34.0522342, 52.52000659999999, 40.3816571, 40.7723679, 37.9100783, 34.0522342, 40.7324194, 34.0522342, -23.5505199, 40.7324194, 34.0522342, 40.7324194, 51.895927, 40.46366700000001, 37.3382082, 37.2321077, 34.0522342, 34.0522342, 42.3600825, 40.7324194, 34.0522342, 42.3600825, 34.0522342, 36.1176734, 27.6648274, 43.7193953, 6.5243793, 41.3850639, 41.153332, 40.7324194, 41.9027835, 41.9027835, 40.7324194, 40.7324194, 40.7324194, 36.9308935, 34.0522342, 19.0414398, 40.7324194, 52.132633, 34.0522342, 40.7324194, 40.7324194, 34.0522342, 46.818188, 40.7324194, 9.5835546, 46.3115038, 36.1176734, 40.7324194, 52.52000659999999, 53.5510846, 40.7324194, 40.7324194, 40.7324194, 40.7324194, 37.2321077, 40.7324194, 36.1019219, 38.6270025, 40.7324194, 40.5795317, 6.5243793, 52.3555177, 40.7324194, 40.7324194, 34.0522342, 40.7324194, 40.7324194, 40.1570707, 37.3382082, 37.09024, 52.52000659999999, 25.2048493, 46.862496, 40.7324194, 40.7324194, 37.0373074, 6.5243793, 40.0149856, 40.7324194, 52.3555177, 40.7324194, 52.3555177, 42.1103041, 39.0457549, 40.7324194, 52.3555177, 23.810332, 41.153332, 51.7520209, 40.7324194, 6.5243793, 52.52000659999999, 37.2321077, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 20.593684, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 37.09024, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 40.0421401, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 40.7324194, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 40.7324194, 51.7520209, 28.0105745, 28.0105745, 28.0105745, 40.7324194, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 20.593684, 40.7324194, 28.0105745, 40.7324194, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 28.0105745, 40.7324194, 40.46366700000001]
# Initialaizing Lat and Lng]
longitude = [-73.98816670000001, -73.98816670000001, -95.9216538, -73.98816670000001, -73.98816670000001, -73.98816670000001, 13.404954, -73.98816670000001, 10.451526, -73.98816670000001, -116.419389, -73.98816670000001, -73.98816670000001, -73.98816670000001, -88.0132958, -73.98816670000001, -95.8818853, -3.74922, -73.98816670000001, -73.98816670000001, -116.419389, -73.98816670000001, -73.98816670000001, 6.6827638, -118.2436849, -95.9216538, -73.98816670000001, 77.10249019999999, -73.98816670000001, -122.3320708, -97.1430671, -95.9216538, 102.0977693, -87.6297982, -118.2436849, -116.419389, -97.1430671, -95.7243335, -73.98816670000001, -73.98816670000001, -76.7074571, -97.1430671, 139.7402946, -73.98816670000001, -95.9216538, -93.29334190000002, -95.7243335, 137.2923893, -1.1743197, 78.96288, 78.96288, -0.1277583, 78.96288, -73.98816670000001, -106.346771, 13.2011404, -106.346771, 78.96288, -73.98816670000001, -95.7243335, 139.6980773, 78.96288, 77.10249019999999, -73.98816670000001, -95.61636639999999, -73.98816670000001, -73.98816670000001, -95.712891, -95.712891, 6.235976, -73.98816670000001, -81.5157535, -73.98816670000001, -73.98816670000001, -80.244216, -73.98816670000001, -94.7605437, -73.98816670000001, 18.0685808, 139.7537719, -122.6750261, -121.9552356, 18.643501, -73.98816670000001, 5.36978, -73.98816670000001, 8.227511999999999, -73.98816670000001, -73.98816670000001, -1.1743197, -78.8783689, -73.98816670000001, -115.1398296, 73.8567437, -73.98816670000001, -46.63330939999999, -93.2650108, -75.1652215, -95.61636639999999, 2.3522219, -95.9216538, -73.98816670000001, 20.168331, 50.54997299999999, -73.98816670000001, -77.6088465, 5.291265999999999, 80.77179699999999, -0.1277583, -0.1277583, -0.1277583, -73.98816670000001, -0.1277583, -0.1277583, -0.1277583, -0.1277583, -122.1660756, -0.1277583, -0.1277583, -0.1277583, -73.98816670000001, -71.10973349999999, -73.9712488, -95.7243335, 13.404954, -73.98816670000001, -1.1743197, -73.4298073, 10.451526, -95.712891, -73.98816670000001, -73.98816670000001, 37.906193, -95.61636639999999, -95.61636639999999, 10.451526, -80.1917902, 10.451526, -94.7605437, -87.771964, -73.98816670000001, -79.3831843, -73.98816670000001, -73.98816670000001, 78.96288, -3.435973, -1.1743197, 72.8776559, 4.9041389, -73.98816670000001, -73.98816670000001, -73.98816670000001, 13.404954, -0.1277583, -0.1277583, -0.1277583, -73.98816670000001, 78.486671, -73.98816670000001, -79.6248197, -73.98816670000001, 73.8567437, -73.98816670000001, -73.98816670000001, -74.00300829999999, -1.1743197, 121.0244452, -74.00237589999999, -77.0368707, -73.98816670000001, -104.990251, -73.98816670000001, -73.98816670000001, -73.98816670000001, -122.4194155, -76.49451049999999, -76.49451049999999, -3.17909, -73.98816670000001, 13.404954, -73.98816670000001, -3.7037902, -97.6935397, -73.98816670000001, -86.7816016, -73.98816670000001, -4.102247999999999, 37.906193, -73.98816670000001, 73.0296625, -73.98816670000001, 77.10249019999999, 103.819836, 10.451526, -73.98816670000001, -73.98816670000001, -74.0059728, -1.1743197, 72.5713621, -73.98816670000001, -93.8454821, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -1.1743197, -117.8311428, 2.1734035, -73.98816670000001, -73.98816670000001, 8.227511999999999, 4.3517211, -76.49451049999999, -76.49451049999999, -73.98816670000001, -73.98816670000001, -73.98816670000001, 133.775136, -73.98816670000001, -95.8677414, -95.8677414, -80.782127, 36.8219462, -73.98816670000001, 60.64540859999999, -116.419389, 88.36389500000001, -95.712891, -73.8183804, -73.98816670000001, 10.451526, -90.0981821, -4.2026458, -73.98816670000001, -73.98816670000001, -75.4656511, 11.5819805, 10.451526, -73.98816670000001, -75.4656511, -77.7967676, -73.98816670000001, -85.6704798, 8.675277, -3.7037902, 77.10249019999999, 37.906193, -73.98816670000001, -73.98816670000001, -73.98816670000001, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, -95.61636639999999, -95.61636639999999, -95.61636639999999, -95.61636639999999, -95.61636639999999, -95.61636639999999, -95.61636639999999, -95.61636639999999, -95.61636639999999, -95.61636639999999, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -76.64127119999999, -73.98816670000001, -73.98816670000001, -95.61636639999999, -95.61636639999999, -95.61636639999999, -73.98816670000001, 4.9041389, -73.98816670000001, -73.98816670000001, 78.96288, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, 78.96288, -95.3698028, -112.0740373, -112.0740373, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -93.63191309999999, -95.9840012, -73.98816670000001, -95.8818853, -3.7037902, 77.10249019999999, 78.0727234, -73.98816670000001, 74.21793260000001, -95.712891, -95.7243335, -73.98816670000001, 10.451526, -73.98816670000001, -95.7243335, -89.5925857, 144.9630576, -73.98816670000001, -95.7243335, -97.7430608, -97.7430608, -97.7430608, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, 73.22068, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, 8.227511999999999, 140.6647405, -95.7243335, -95.791331, -95.61636639999999, 78.96288, -95.9216538, 78.96288, 10.451526, -73.98816670000001, -95.7243335, -95.61636639999999, -73.98816670000001, -73.98816670000001, -73.98816670000001, -74.0059728, -74.0059728, 8.675277, -97.2778925, -73.98816670000001, 2.1734035, 8.675277, -73.98816670000001, 8.675277, 8.675277, -73.98816670000001, -93.8454821, -0.1277583, -0.1277583, -73.98816670000001, 2.213749, -106.346771, -4.102247999999999, 8.675277, -86.1089301, 5.36978, -94.663136, -73.98816670000001, -95.99277500000001, -73.98816670000001, -73.98816670000001, 80.77179699999999, -73.98816670000001, -73.98816670000001, 18.9115964, -95.7243335, -95.61636639999999, 13.404954, -73.98816670000001, -118.2436849, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, 3.3792057, -73.98816670000001, -121.8863286, -73.98816670000001, 3.36005, 27.142826, 103.846656, -73.98816670000001, -118.2436849, 3.3792057, -75.4656511, -80.1917902, -87.771964, -94.2229437, -102.552784, 2.25929, 8.227511999999999, -118.2436849, 2.3522219, 8.541694, -95.9216538, 5.291265999999999, 78.96288, 6.1431577, -77.0368707, -3.435973, 8.675277, -95.712891, -79.3741862, -75.4656511, -95.7243335, -111.8910474, -79.3741862, -95.712891, -73.98816670000001, -122.4088726, 12.5782654, 3.3792057, -122.4088726, -75.1652215, -99.133208, -73.98816670000001, -73.98816670000001, -95.9216538, -73.98816670000001, -95.99277500000001, -73.98816670000001, 80.77179699999999, -73.98816670000001, -95.712891, -73.98816670000001, -73.98816670000001, -73.98816670000001, -122.4088726, -80.5204096, -73.98816670000001, -73.98816670000001, -73.98816670000001, 80.77179699999999, 3.3792057, -73.98816670000001, -73.98816670000001, 103.846656, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -75.4656511, -75.4656511, 13.404954, 3.3792057, -99.133208, 13.404954, -73.98816670000001, 37.6172999, 8.6821267, -0.1277583, -73.98816670000001, -73.98816670000001, -73.98816670000001, 13.404954, 10.451526, -73.98816670000001, 80.77179699999999, -73.98816670000001, 13.404954, 34.851612, -73.98816670000001, -73.98816670000001, -76.3888087, -73.98816670000001, -73.98816670000001, 13.404954, -73.98816670000001, -73.98816670000001, 10.451526, 11.2558136, -73.98816670000001, -95.7243335, -76.6074575, -73.98816670000001, -73.98816670000001, -79.3831843, -73.98816670000001, -95.9216538, -83.0790865, -83.0790865, 8.227511999999999, -73.98816670000001, 101.7129232, -73.98816670000001, -95.7243335, -77.49677319999999, -73.98816670000001, -83.0790865, -73.98816670000001, 80.77179699999999, -73.98816670000001, -72.5781404, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, 13.404954, -106.650422, -73.98816670000001, -76.49451049999999, -75.39022059999999, -86.25198979999999, 3.3792057, -73.98816670000001, -73.98816670000001, -121.8863286, -73.98816670000001, -95.9216538, 80.77179699999999, -73.98816670000001, -73.98816670000001, -73.98816670000001, 13.404954, -3.74922, -66.58973, -73.98816670000001, -95.61636639999999, -95.712891, -118.2436849, -95.712891, 8.227511999999999, -118.2436849, -95.712891, -95.9216538, -95.712891, 103.846656, 10.451526, -95.712891, -79.9353141, 2.1734035, -95.712891, -115.1398296, -95.7243335, -76.64127119999999, -71.10973349999999, -73.98816670000001, -118.2436849, -118.2436849, -121.8863286, -73.98816670000001, -79.9353141, 103.846656, 103.846656, -73.98816670000001, 77.10249019999999, -112.0740373, -95.7243335, -117.1610838, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -72.5781404, -73.98816670000001, -79.9353141, -105.084423, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -93.8454821, -73.98816670000001, -3.435973, 80.77179699999999, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -95.7243335, -73.98816670000001, -73.98816670000001, -73.98816670000001, 13.404954, 103.846656, -73.98816670000001, -74.072092, 103.846656, -73.98816670000001, -73.98816670000001, -8.629105299999999, -73.98816670000001, -73.98816670000001, -8.629105299999999, -73.98816670000001, -4.102247999999999, 78.96288, -73.98816670000001, -95.7243335, -73.98816670000001, -73.98816670000001, 103.846656, -73.98816670000001, -121.8863286, -73.98816670000001, 20.168331, -78.6568942, -73.98816670000001, -95.7243335, -73.98816670000001, 3.3792057, -73.98816670000001, -97.1430671, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -97.1430671, -73.98816670000001, 12.56738, -73.98816670000001, 103.846656, -0.1277583, -0.1277583, -116.5765035, -73.98816670000001, -76.64127119999999, 12.56738, -73.98816670000001, -119.4179324, 3.3792057, -74.0059728, -73.98816670000001, -73.98816670000001, 12.56738, -73.98816670000001, 10.451526, 12.56738, -74.0059728, -73.98816670000001, -73.98816670000001, -95.9216538, -118.2436849, -118.2436849, -73.98816670000001, 2.213749, -73.98816670000001, -76.64127119999999, 103.846656, 2.1734035, 80.2707184, -118.2436849, 8.6821267, -73.98816670000001, -118.2436849, 3.3792057, -3.74922, -121.8863286, -71.1934016, -71.1934016, -73.98816670000001, -93.097702, 77.2090212, -76.3888087, -73.98816670000001, -73.98816670000001, -3.74922, -73.98816670000001, -1.3080629, 80.2707184, -73.98816670000001, -73.9888338, -118.2436849, -73.98816670000001, 73.22068, -0.57918, 8.6821267, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -71.1934016, -118.2436849, -73.98816670000001, -86.158068, 8.675277, -71.1934016, 103.846656, 174.885971, 10.451526, -75.4656511, 103.846656, 8.227511999999999, -73.98816670000001, -71.1934016, -76.64127119999999, -78.6381787, -73.98816670000001, -73.98816670000001, -78.4678382, -95.7243335, -81.6326234, -73.98816670000001, -73.98816670000001, 77.2090212, -73.98816670000001, -73.98816670000001, -73.98816670000001, 3.3792057, 133.775136, -73.98816670000001, -73.98816670000001, 3.2246995, -103.3496092, -73.98816670000001, 3.3792057, -73.98816670000001, -73.98816670000001, 2.3522219, 5.36978, 78.486671, -73.98816670000001, 103.846656, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, 3.3792057, 103.846656, 12.6761041, 77.5945627, 77.5945627, -73.98816670000001, -73.98816670000001, 2.1734035, 15.8049558, -73.98816670000001, 130.2920243, 12.6761041, -72.5781404, -95.712891, -95.712891, -95.712891, -73.98816670000001, -95.61636639999999, -73.98816670000001, -73.98816670000001, -94.7605437, -73.98816670000001, 103.846656, -118.2436849, 13.404954, -74.5654411, -74.6049061, -122.0651819, -118.2436849, -73.98816670000001, -118.2436849, -46.63330939999999, -73.98816670000001, -118.2436849, -73.98816670000001, 0.8918740000000001, -3.74922, -121.8863286, -95.7243335, -118.2436849, -118.2436849, -71.0588801, -73.98816670000001, -118.2436849, -71.0588801, -118.2436849, -95.9216538, -81.5157535, -79.6372743, 3.3792057, 2.1734035, 20.168331, -73.98816670000001, 12.4963655, 12.4963655, -73.98816670000001, -73.98816670000001, -73.98816670000001, -93.8454821, -118.2436849, -98.2062727, -73.98816670000001, 5.291265999999999, -118.2436849, -73.98816670000001, -73.98816670000001, -118.2436849, 8.227511999999999, -73.98816670000001, 6.5463156, -72.5781404, -95.9216538, -73.98816670000001, 13.404954, 9.9936819, -73.98816670000001, -73.98816670000001, -73.98816670000001, -73.98816670000001, -95.7243335, -73.98816670000001, -95.8677414, -90.19940419999999, -73.98816670000001, -74.1502007, 3.3792057, -1.1743197, -73.98816670000001, -73.98816670000001, -118.2436849, -73.98816670000001, -73.98816670000001, -75.4656511, -121.8863286, -95.712891, 13.404954, 55.2707828, 103.846656, -73.98816670000001, -73.98816670000001, -95.61636639999999, 3.3792057, -105.2705456, -73.98816670000001, -1.1743197, -73.98816670000001, -1.1743197, -88.03424000000001, -76.64127119999999, -73.98816670000001, -1.1743197, 90.4125181, 20.168331, -1.2577263, -73.98816670000001, 3.3792057, 13.404954, -95.7243335, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, 78.96288, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -95.712891, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -76.3888087, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -73.98816670000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -73.98816670000001, -1.2577263, -82.57731930000001, -82.57731930000001, -82.57731930000001, -73.98816670000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, 78.96288, -73.98816670000001, -82.57731930000001, -73.98816670000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -82.57731930000001, -73.98816670000001, -3.74922]
print('''
    Latitude: {0}
    Longitude: {1}
    '''.format(latitude, longitude))


    Latitude: [40.7324194, 40.7324194, 36.1176734, 40.7324194, 40.7324194, 40.7324194, 52.52000659999999, 40.7324194, 51.165691, 40.7324194, 38.8026097, 40.7324194, 40.7324194, 40.7324194, 44.5133188, 40.7324194, 36.0637737, 40.46366700000001, 40.7324194, 40.7324194, 38.8026097, 40.7324194, 40.7324194, 46.5088803, 34.0522342, 36.1176734, 40.7324194, 28.7040592, 40.7324194, 47.6062095, 32.844017, 36.1176734, 14.9798997, 41.8781136, 34.0522342, 38.8026097, 32.844017, 37.2321077, 40.7324194, 40.7324194, 37.2707022, 32.844017, 35.70145979999999, 40.7324194, 36.1176734, 37.1613814, 37.2321077, 35.0182505, 52.3555177, 20.593684, 20.593684, 51.5073509, 20.593684, 40.7324194, 56.130366, 52.6320478, 56.130366, 20.593684, 40.7324194, 37.2321077, 35.6414529, 20.593684, 28.7040592, 40.7324194, 37.0373074, 40.7324194, 40.7324194, 37.09024, 37.09024, 44.09219299999999, 40.7324194, 27.6648274, 40.7324194, 40.7324194, 36.09985959999999, 40.7324194, 39.101825, 40.7324194, 59.32932349999999, 35.6940309

In [95]:
lat_lng_df=pd.DataFrame()
lat_lng_df['id_str'] = new_df['id_str']
lat_lng_df['latitude']=pd.DataFrame(latitude)
lat_lng_df['longitude']=pd.DataFrame(longitude)
lat_lng_df

,id_str,latitude,longitude
0,1337834038262829056,40.732419,-73.988167
1,1337834046794117120,40.732419,-73.988167
2,1337834048690024453,36.117673,-95.921654
3,1337834049503629312,40.732419,-73.988167
4,1337834059930755075,40.732419,-73.988167
...,...,...,...
118470,1338585941032787969,NaN,NaN
118471,1338585945063493632,NaN,NaN
118472,1338585969390456840,NaN,NaN
118473,1338585980115234825,NaN,NaN


In [96]:
final_df=combined_df.merge(lat_lng_df, on='id_str')
final_df.head()

,id_str,name,screen_name,location,description,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at,text,lang,latitude,longitude
0,1337834048690024453,Jessica shoonover,Women_who_code,bootcamp,24 🧬 ~ Javascript ❤️ ~ Mern Stack Developer ~ ...,1665,23,30,455,199485,Mon Jun 22 13:23:34 +0000 2020,RT @SourabhSKatoch: Daily Life as a Data Scien...,en,36.117673,-95.921654
1,1337834060350164993,FemTech_🖥👩🏽‍💻,femtech_,"Berlin, Deutschland",I'm a friendly bot retweeting female developer...,16960,2,186,87,950486,Tue Jun 25 14:36:58 +0000 2019,RT @SourabhSKatoch: Daily Life as a Data Scien...,en,52.520007,13.404954
2,1337834062610886662,HubOfML,hubofml,Germany,"Once a month, you'll get a newsletter containi...",6850,12,150,32093,298035,Mon Feb 16 15:33:55 +0000 2015,RT @SourabhSKatoch: Daily Life as a Data Scien...,en,51.165691,10.451526
3,1337834065240731655,NerdyFlow,nerdyflow,"Nevada, USA",Draw your imagination. The leading graphic tab...,12703,2728,47,270716,567214,Tue Mar 26 04:19:19 +0000 2019,RT @Sprite4it: Best 4 Medical Google Slides Te...,en,38.802610,-116.419389
4,1337834110685995012,Gatuaprof,gatuaprof,Greenbay,"Biomedical scientists-molecular biology, virol...",217,601,1,750,249,Sun Apr 28 13:13:33 +0000 2019,@NjeriAquila @Ruthnanje @RitahNabunje @EvaAkurut,und,44.513319,-88.013296


In [97]:
final_df=final_df.dropna(subset=['latitude', 'longitude'])

In [98]:
final_df

,id_str,name,screen_name,location,description,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at,text,lang,latitude,longitude
0,1337834048690024453,Jessica shoonover,Women_who_code,bootcamp,24 🧬 ~ Javascript ❤️ ~ Mern Stack Developer ~ ...,1665,23,30,455,199485,Mon Jun 22 13:23:34 +0000 2020,RT @SourabhSKatoch: Daily Life as a Data Scien...,en,36.117673,-95.921654
1,1337834060350164993,FemTech_🖥👩🏽‍💻,femtech_,"Berlin, Deutschland",I'm a friendly bot retweeting female developer...,16960,2,186,87,950486,Tue Jun 25 14:36:58 +0000 2019,RT @SourabhSKatoch: Daily Life as a Data Scien...,en,52.520007,13.404954
2,1337834062610886662,HubOfML,hubofml,Germany,"Once a month, you'll get a newsletter containi...",6850,12,150,32093,298035,Mon Feb 16 15:33:55 +0000 2015,RT @SourabhSKatoch: Daily Life as a Data Scien...,en,51.165691,10.451526
3,1337834065240731655,NerdyFlow,nerdyflow,"Nevada, USA",Draw your imagination. The leading graphic tab...,12703,2728,47,270716,567214,Tue Mar 26 04:19:19 +0000 2019,RT @Sprite4it: Best 4 Medical Google Slides Te...,en,38.802610,-116.419389
4,1337834110685995012,Gatuaprof,gatuaprof,Greenbay,"Biomedical scientists-molecular biology, virol...",217,601,1,750,249,Sun Apr 28 13:13:33 +0000 2019,@NjeriAquila @Ruthnanje @RitahNabunje @EvaAkurut,und,44.513319,-88.013296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,1337858503151661058,PyScale,PyScaleLLC,"Town 'n' Country, FL","Edward Turner, a polyglot technologist, is the...",139,182,4,150,4703,Sun Sep 20 14:23:07 +0000 2020,RT @Necio_news: Global Big Data Conference #De...,en,28.010575,-82.577319
636,1337858503906615306,PyScale,PyScaleLLC,"Town 'n' Country, FL","Edward Turner, a polyglot technologist, is the...",139,182,4,150,4704,Sun Sep 20 14:23:07 +0000 2020,RT @Ronald_vanLoon: The Ultimate Cheatsheet Fo...,en,28.010575,-82.577319
637,1337858504657395721,PyScale,PyScaleLLC,"Town 'n' Country, FL","Edward Turner, a polyglot technologist, is the...",139,182,4,150,4705,Sun Sep 20 14:23:07 +0000 2020,RT @Rwill235: Excited to announce my Machine L...,en,28.010575,-82.577319
638,1337858505462722563,PyScale,PyScaleLLC,"Town 'n' Country, FL","Edward Turner, a polyglot technologist, is the...",139,182,4,150,4706,Sun Sep 20 14:23:07 +0000 2020,RT @12S_FOR_LEADER: “Technology eliminates the...,en,28.010575,-82.577319


In [101]:
final_df=final_df.rename(columns={'created_at_y': "account_creation_date", 'lang': 'language'})

In [102]:
final_df.head()

,id_str,name,screen_name,location,description,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at,text,language,latitude,longitude
0,1337834048690024453,Jessica shoonover,Women_who_code,bootcamp,24 🧬 ~ Javascript ❤️ ~ Mern Stack Developer ~ ...,1665,23,30,455,199485,Mon Jun 22 13:23:34 +0000 2020,RT @SourabhSKatoch: Daily Life as a Data Scien...,en,36.117673,-95.921654
1,1337834060350164993,FemTech_🖥👩🏽‍💻,femtech_,"Berlin, Deutschland",I'm a friendly bot retweeting female developer...,16960,2,186,87,950486,Tue Jun 25 14:36:58 +0000 2019,RT @SourabhSKatoch: Daily Life as a Data Scien...,en,52.520007,13.404954
2,1337834062610886662,HubOfML,hubofml,Germany,"Once a month, you'll get a newsletter containi...",6850,12,150,32093,298035,Mon Feb 16 15:33:55 +0000 2015,RT @SourabhSKatoch: Daily Life as a Data Scien...,en,51.165691,10.451526
3,1337834065240731655,NerdyFlow,nerdyflow,"Nevada, USA",Draw your imagination. The leading graphic tab...,12703,2728,47,270716,567214,Tue Mar 26 04:19:19 +0000 2019,RT @Sprite4it: Best 4 Medical Google Slides Te...,en,38.802610,-116.419389
4,1337834110685995012,Gatuaprof,gatuaprof,Greenbay,"Biomedical scientists-molecular biology, virol...",217,601,1,750,249,Sun Apr 28 13:13:33 +0000 2019,@NjeriAquila @Ruthnanje @RitahNabunje @EvaAkurut,und,44.513319,-88.013296


# Load data back into database

In [103]:
#connect to mongo
MONGO_HOST = 'mongodb://localhost/twitteranalysis'
client = MongoClient(MONGO_HOST)
# use twitteranalysis database. if it doesn't exist, it will be created
db= client.twitteranalysis
#collection name
twitter_collection = db.twitter_collection

In [104]:
#insert data into targeted database
final_df.reset_index(inplace=True)
data_dict = final_df.to_dict('records')
db.twitter_collection.insert_many(data_dict)

## Additional Analysis

In [115]:
import gmaps
from matplotlib.cm import viridis
from matplotlib.colors import to_hex

In [123]:
df = pd.DataFrame({'screen_name':final_df['screen_name'],'tweet':final_df['text']})
df.fillna(0)
user_location = final_df.to_dict('records')

info_box_template="""<dl>
<dt>Screen Name</dt><dd>{screen_name}</dd>
<dt>Tweet</dt><dd>{text}</dd>
</dl>
"""
locations = final_df[['latitude', 'longitude']]
location_info =[info_box_template.format(**row) for row in user_location]
marker_layer = gmaps.marker_layer(locations, info_box_content=location_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [124]:
df

,screen_name,tweet
0,Women_who_code,RT @SourabhSKatoch: Daily Life as a Data Scien...
1,femtech_,RT @SourabhSKatoch: Daily Life as a Data Scien...
2,hubofml,RT @SourabhSKatoch: Daily Life as a Data Scien...
3,nerdyflow,RT @Sprite4it: Best 4 Medical Google Slides Te...
4,gatuaprof,@NjeriAquila @Ruthnanje @RitahNabunje @EvaAkurut
...,...,...
635,PyScaleLLC,RT @Necio_news: Global Big Data Conference #De...
636,PyScaleLLC,RT @Ronald_vanLoon: The Ultimate Cheatsheet Fo...
637,PyScaleLLC,RT @Rwill235: Excited to announce my Machine L...
638,PyScaleLLC,RT @12S_FOR_LEADER: “Technology eliminates the...
